# Metadata

```yaml
Course:   DS 5001
Module:   08a Lab
Topic:    Using MALLET
Author:   R.C. Alvarado
Date:     23 March 2023

```

# Notes

See Appendix below information on how download and install <a href="https://mimno.github.io/Mallet/topics">MALLET</a>.

# Set Up

In [1]:
data_home = "../data"
local_lib = "../lib"
data_prefix = 'novels'
mallet_data = "./mallet"
OHCO = ['book_id','chap_id','para_num','sent_num','token_num']

In [2]:
max_words = 10000

# For MALLET
num_topics = 20
num_iters = 1000
show_interval = 100

In [3]:
import pandas as pd
import numpy as np

# Import CORPUS

In [4]:
LIB = pd.read_csv(f"{data_home}/{data_prefix}/{data_prefix}-LIB.csv").set_index(OHCO[:1])

In [5]:
CORPUS = pd.read_csv(f"{data_home}/{data_prefix}/{data_prefix}-CORPUS.csv").set_index(OHCO)

In [6]:
CORPUS.head()

pos     term_str
book_id         chap_id para_num sent_num token_num                  
secretadversary 1       0        1        0           DT          the
                                          1          NNP        young
                                          2          NNP  adventurers
                                          3          NNP          ltd
                        1        0        0           JJ        tommy

# Create DOC

In [7]:
def gather_docs(CORPUS, ohco_level, term_col='term_str'):
    OHCO = CORPUS.index.names
    CORPUS[term_col] = CORPUS[term_col].astype('str')
    DOC = CORPUS.groupby(OHCO[:ohco_level])[term_col].apply(lambda x:' '.join(x)).to_frame('doc_str')
    return DOC

In [8]:
DOC = gather_docs(CORPUS, 2)

In [9]:
DOC['n_tokens'] = DOC.doc_str.apply(lambda x: len(x.split()))

In [10]:
DOC

doc_str  \
book_id    chap_id                                                      
adventures 1        a scandal in bohemia i to sherlock holmes she ...   
           2        the red headed league i had called upon my fri...   
           3        a case of identity my dear fellow said sherloc...   
           4        the boscombe valley mystery we were seated at ...   
           5        the five orange pips when i glance over my not...   
...                                                               ...   
udolpho    54       vi unnatural deeds do breed unnatural troubles...   
           55       vii but in these cases we still have judgment ...   
           56       viii then fresh tears stood on her cheek as do...   
           57       ix now my task is smoothly done i can fly or i...   
usher      1        fall of the house of usher son coeur est un lu...   

                    n_tokens  
book_id    chap_id            
adventures 1            8608  
           2            9197  
           3            7020  
           4            9685  
           5            7365  
...                      ...  
udolpho    54           5605  
           55           4164  
           56           2522  
           57            977  
usher      1            7212  

[320 rows x 2 columns]

## Dump corpus to CSV file

In [11]:
mallet_corpus = DOC.join(LIB)[['doc_str','author_id']]
mallet_corpus.columns = 'doc_content doc_label'.split()
mallet_corpus[['doc_label','doc_content']].to_csv('corpus/novels-corpus.csv', index=False)

## MALLET Time

### Show MALLET options

In [12]:
mallet_home = "/Users/rca2t1/opt/mallet/bin"

In [13]:
! {mallet_home}/mallet 

Unrecognized command: 
Mallet 2.0 commands: 

  import-dir         load the contents of a directory into mallet instances (one per file)
  import-file        load a single file into mallet instances (one per line)
  import-svmlight    load SVMLight format data files into Mallet instances
  info               get information about Mallet instances
  train-classifier   train a classifier from Mallet data files
  classify-dir       classify data from a single file with a saved classifier
  classify-file      classify the contents of a directory with a saved classifier
  classify-svmlight  classify data from a single file in SVMLight format
  train-topics       train a topic model from Mallet data files
  infer-topics       use a trained topic model to infer topics for new documents
  evaluate-topics    estimate the probability of new documents under a trained model
  prune              remove features based on frequency or information gain
  split              divide data into testing, tr

### Import corpus

In [14]:
!{mallet_home}/mallet import-file --input corpus/novels-corpus.csv --output mallet/novels-corpus.mallet --keep-sequence TRUE

### Train topics

In [15]:
!{mallet_home}/mallet train-topics --input mallet/novels-corpus.mallet --num-topics {num_topics} --num-iterations {num_iters} \
--output-doc-topics mallet/novels-doc-topics.txt \
--output-topic-keys mallet/novels-topic-keys.txt \
--word-topic-counts-file mallet/novels-word-topic-counts-file.txt \
--topic-word-weights-file mallet/novels-topic-word-weights-file.txt \
--xml-topic-report mallet/novels-topic-report.xml \
--xml-topic-phrase-report mallet/novels-topic-phrase-report.xml \
--show-topics-interval {show_interval} \
--use-symmetric-alpha false  \
--optimize-interval 100 \
--diagnostics-file mallet/novels-diagnostics.xml

Mallet LDA: 20 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 15717
total tokens: 1164070
<10> LL/token: -9.08018
<20> LL/token: -8.59597
<30> LL/token: -8.41396
<40> LL/token: -8.31237
<50> LL/token: -8.24817
<60> LL/token: -8.20836
<70> LL/token: -8.17174
<80> LL/token: -8.15141
<90> LL/token: -8.12778

0	0.25	the manfred said thou not this project isabella gutenberg and his matilda thy with lord for work theodore thee any 
1	0.25	the and was had been from which with that some into his have where even part years doubt possible house 
2	0.25	the and his him said sir they lord that their with them all scrooge was were for baron philip father 
3	0.25	that and you the was his holmes had not but have very for would are him yes said all there 
4	0.25	and that was but not had she could her might father for when should own day before did after only 
5	0.25	the you and tuppence tommy was that said but dont she for her with sir they know julius well what 
6	0.25	the and that t

# Appendix: README

<a href="https://mimno.github.io/Mallet/topics">Download MALLET here</a> | <a href="https://github.com/ontoligent/mazo">Mazo, a wrapper around MALLET to organize its output.</a>

[![Build Status](https://travis-ci.com/MNCC/Mallet.svg?branch=master)](https://travis-ci.com/MNCC/Mallet)
[![codecov](https://codecov.io/gh/MNCC/Mallet/branch/master/graph/badge.svg)](https://codecov.io/gh/MNCC/Mallet)

Mallet
======

Website: https://mimno.github.io/Mallet/

MALLET is a Java-based package for statistical natural language processing, document classification, clustering, topic modeling, information extraction, and other machine learning applications to text.

MALLET includes sophisticated tools for document classification: efficient routines for converting text to "features", a wide variety of algorithms (including Naïve Bayes, Maximum Entropy, and Decision Trees), and code for evaluating classifier performance using several commonly used metrics.

In addition to classification, MALLET includes tools for sequence tagging for applications such as named-entity extraction from text. Algorithms include Hidden Markov Models, Maximum Entropy Markov Models, and Conditional Random Fields. These methods are implemented in an extensible system for finite state transducers.

Topic models are useful for analyzing large collections of unlabeled text. The MALLET topic modeling toolkit contains efficient, sampling-based implementations of Latent Dirichlet Allocation, Pachinko Allocation, and Hierarchical LDA.

Many of the algorithms in MALLET depend on numerical optimization. MALLET includes an efficient implementation of Limited Memory BFGS, among many other optimization methods.

In addition to sophisticated Machine Learning applications, MALLET includes routines for transforming text documents into numerical representations that can then be processed efficiently. This process is implemented through a flexible system of "pipes", which handle distinct tasks such as tokenizing strings, removing stopwords, and converting sequences into count vectors.

An add-on package to MALLET, called GRMM, contains support for inference in general graphical models, and training of CRFs with arbitrary graphical structure.

## Installation

To build a Mallet 2.0 development release, you must have the Apache ant build tool installed. From the command prompt, first change to the mallet directory, and then type
`ant`

If `ant` finishes with `"BUILD SUCCESSFUL"`, Mallet is now ready to use.

If you would like to deploy Mallet as part of a larger application, it is helpful to create a single ".jar" file that contains all of the compiled code. Once you have compiled the individual Mallet class files, use the command:
`ant jar`

This process will create a file "mallet.jar" in the "dist" directory within Mallet.

## Usage

Once you have installed Mallet you can use it using the following command:
```
bin/mallet [command] --option value --option value ...
```
Type `bin/mallet` to get a list of commands, and use the option `--help` with any command to get a description of valid options.

For details about the commands please visit the API documentation and website at: https://mimno.github.io/Mallet/


## List of Algorithms

* Topic Modelling
  * LDA
  * Parallel LDA
  * DMR LDA
  * Hierarchical LDA
  * Labeled LDA
  * Polylingual Topic Model
  * Hierarchical Pachinko Allocation Model (PAM)
  * Weighted Topic Model
  * LDA with integrated phrase discovery
  * Word Embeddings (word2vec) using skip-gram with negative sampling
* Classification
  * AdaBoost
  * Bagging
  * Winnow
  * C45 Decision Tree
  * Ensemble Trainer
  * Maximum Entropy Classifier (Multinomial Logistic Regression)
  * Naive Bayes
  * Rank Maximum Entropy Classifier
  * Posterior Regularization Auxiliary Model
* Clustering
  * Greedy Agglomerative
  * Hill Climbing
  * K-Means
  * K-Best
* Sequence Prediction Models
  * Conditional Random Fields
  * Maximum Entropy Markov Models
  * Hidden Markov Models
  * Semi-Supervised Sequence Prediction Models
* Linear Regression



